In [1]:
from __future__ import division
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from difflib import SequenceMatcher

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

#import decisiontreeclassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
#import logisticregression classifier
from sklearn.linear_model import LogisticRegression
#import statsmodels.api as sm
#import knn classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

#for validating your classification model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# feature selection
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

import re
import requests
from lxml import html
import time

In [2]:
df=pd.read_csv('df2_all.csv',low_memory=False)
df.loc[df['reporter_country'].isnull(), 'reporter_country']='country not specified'
df=df.drop([0]).reset_index(drop=True)
df.head()

,primaryid,drug_seq,caseid,indi_pt,drugname,age,gndr_cod,wt,reporter_country,de,lt,ho,ds,ca,ri,ot,pt,drugbank_ID
0,100033001,1.0,10003300.0,osteoporosis,boniva,77.0,0.0,NaN,us,0.0,0.0,0.0,1.0,0.0,0.0,0.0,arthralgia,DB00710
1,100033001,1.0,10003300.0,osteoporosis,boniva,77.0,0.0,NaN,us,0.0,0.0,0.0,1.0,0.0,0.0,0.0,headache,DB00710
2,100033062,1.0,10003306.0,product used for unknown indication,letairis,68.0,0.0,NaN,us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cough,DB06403
3,100033072,2.0,10003307.0,connective tissue disorder,letairis,NaN,NaN,NaN,country not specified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rhinorrhoea,DB06403
4,100033083,2.0,10003308.0,systemic sclerosis,letairis,59.0,0.0,NaN,us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unevaluable event,DB06403


In [3]:
df['drug_page']="https://www.drugbank.ca/drugs/"+df['drugbank_ID']
df.head()

,primaryid,drug_seq,caseid,indi_pt,drugname,age,gndr_cod,wt,reporter_country,de,lt,ho,ds,ca,ri,ot,pt,drugbank_ID,drug_page
0,100033001,1.0,10003300.0,osteoporosis,boniva,77.0,0.0,NaN,us,0.0,0.0,0.0,1.0,0.0,0.0,0.0,arthralgia,DB00710,https://www.drugbank.ca/drugs/DB00710
1,100033001,1.0,10003300.0,osteoporosis,boniva,77.0,0.0,NaN,us,0.0,0.0,0.0,1.0,0.0,0.0,0.0,headache,DB00710,https://www.drugbank.ca/drugs/DB00710
2,100033062,1.0,10003306.0,product used for unknown indication,letairis,68.0,0.0,NaN,us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cough,DB06403,https://www.drugbank.ca/drugs/DB06403
3,100033072,2.0,10003307.0,connective tissue disorder,letairis,NaN,NaN,NaN,country not specified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,rhinorrhoea,DB06403,https://www.drugbank.ca/drugs/DB06403
4,100033083,2.0,10003308.0,systemic sclerosis,letairis,59.0,0.0,NaN,us,0.0,0.0,0.0,0.0,0.0,0.0,0.0,unevaluable event,DB06403,https://www.drugbank.ca/drugs/DB06403


In [9]:
dr_bank=pd.DataFrame(df['drugbank_ID'].unique(), columns={'drugbank_ID'})
dr_bank=dr_bank[dr_bank['drugbank_ID']!='None']
dr_bank=dr_bank.dropna().reset_index(drop=True)
dr_bank['target']=None
dr_bank['pathway']=None
dr_bank['PubID']=None
dr_bank.head()

,drugbank_ID,target,pathway,PubID
0,DB00710,None,None,None
1,DB06403,None,None,None
2,DB00678,None,None,None
3,DB06650,None,None,None
4,DB00916,None,None,None


In [34]:
webpage="https://www.drugbank.ca/drugs/DB06403"
r = requests.get(webpage)
data = html.fromstring(r.text)


In [35]:
if "PubChem Compound" in data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/text()"):
    print ("dd")

#if data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/text()").index("PubChem Compound"):
#    print ("dd")

dd


In [24]:
data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/a/text()")

[]

In [16]:
if data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/text()")=='PubChem Compound':
    print (data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dd/a/text()"))
#data.xpath("//html/body/main/div/div[4]/dl[4]/dd[3]/dl/dd[2]/a/text()")

In [36]:
for i in range(len(dr_bank)):
    webpage="https://www.drugbank.ca/drugs/" + dr_bank['drugbank_ID'][i]
    r = requests.get(webpage)
    data = html.fromstring(r.text)
    dr_bank['target'][i] = data.xpath("//*[@id='drug-moa-target-table']/tbody/tr/td[1]/a/text()")
    dr_bank['pathway'][i]= data.xpath("//*[@id='drug-pathways']/tbody/tr/td[1]/a/text()")
    if "PubChem Compound" in data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/text()"):
        j=data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dt/text()").index("PubChem Compound")
        dr_bank['PubID'][i]  = data.xpath("/html/body/main/div/div[4]/dl[4]/dd[3]/dl/dd/a/text()")[j]
    time.sleep(0.5)

In [53]:
dr_bank_copy=dr_bank.copy()

In [54]:
dr_bank_copy.head()

,drugbank_ID,target,pathway,PubID
0,DB00710,"[Farnesyl pyrophosphate synthase, Hydroxylapat...",[Ibandronate Action Pathway],60852
1,DB06403,"[Endothelin-1 receptor, Endothelin B receptor]",[],6918493
2,DB00678,[Type-1 angiotensin II receptor],[Losartan Action Pathway],3961
3,DB06650,[B-lymphocyte antigen CD20],[],[]
4,DB00916,"[DNA, Oxygen-insensitive NADPH nitroreductase,...",[],4173


In [55]:
s=dr_bank_copy.apply(lambda x: pd.Series(x['target']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'target'
dr_bank_copy=dr_bank_copy.drop('target', axis=1).join(s)
d=dr_bank_copy.apply(lambda x: pd.Series(x['pathway']),axis=1).stack().reset_index(level=1, drop=True)
d.name = 'pathway'
dr_bank_copy=dr_bank_copy.drop('pathway', axis=1).join(d)
dr_bank_copy.head()

,drugbank_ID,PubID,target,pathway
0,DB00710,60852,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
0,DB00710,60852,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
0,DB00710,60852,Hydroxylapatite,Ibandronate Action Pathway
0,DB00710,60852,Hydroxylapatite,Ibandronate Action Pathway
1,DB06403,6918493,Endothelin-1 receptor,NaN


In [56]:
len(dr_bank_copy)

1117123

In [43]:
dr_bank_copy.head()

,drugbank_ID,PubID,target,pathway
0,DB00710,60852,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
0,DB00710,60852,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
0,DB00710,60852,Hydroxylapatite,Ibandronate Action Pathway
0,DB00710,60852,Hydroxylapatite,Ibandronate Action Pathway
1,DB06403,6918493,Endothelin-1 receptor,NaN


In [48]:
dr_bank_copy=dr_bank_copy.reset_index(drop=True)

In [51]:
dr_bank_copy=dr_bank_copy.loc[dr_bank_copy.astype(str).drop_duplicates().index]

In [52]:
len(dr_bank_copy)

26828

In [57]:
df2=df.copy()

In [60]:
df2=df2.merge(dr_bank_copy, on='drugbank_ID', how='inner')
df2.head()

MemoryError: 

In [61]:
df2.to_csv('df_stuf.csv', index=False, encoding='utf8')

In [64]:
df2['indi_pt'].value_counts()

product used for unknown indication             16395772
drug use for unknown indication                  3309795
pulmonary arterial hypertension                  2518618
rheumatoid arthritis                             2415183
osteoporosis                                     2366444
multiple sclerosis                               1348548
osteopenia                                        976763
depression                                        916524
insomnia                                          882300
pain                                              847830
pulmonary hypertension                            819658
contraception                                     767726
prophylaxis                                       751361
schizophrenia                                     735041
psoriasis                                         729096
anaemia                                           692212
crohn's disease                                   648928
diabetes mellitus              

In [65]:
df3=df2[['indi_pt','drugname','age','gndr_cod','wt','pt','target','pathway']]

In [ ]:
del(df2)

In [182]:
df3.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
60,osteopenia,boniva,59.0,0.0,48.534344,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
61,osteopenia,boniva,59.0,0.0,48.534344,0,Hydroxylapatite,Ibandronate Action Pathway
262,prophylaxis,boniva,61.0,0.0,95.240000,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
263,prophylaxis,boniva,61.0,0.0,95.240000,0,Hydroxylapatite,Ibandronate Action Pathway
306,osteopenia,boniva,58.0,0.0,63.500000,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [69]:
len(df3)

67262885

In [70]:
len(df3.dropna())

10755993

In [178]:
df4=df3.dropna().reset_index(drop=True)

In [167]:
df4.to_csv('df_dr_reac.csv', index=False)

In [6]:
df4=pd.read_csv('df_dr_reac.csv')
df4.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
0,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
1,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Hydroxylapatite,Ibandronate Action Pathway
2,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
3,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Hydroxylapatite,Ibandronate Action Pathway
4,osteoporosis,boniva,72.2,0.0,39.68,vomiting,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [11]:
prostate_canc=df4[df4['indi_pt'].str.contains('prostate')]
len(prostate_canc)

9585

In [13]:
prostate_canc['indi_pt'].unique()

array(['prostate cancer metastatic', 'prostate cancer',
       'prostate cancer stage iv', 'prostate cancer stage ii',
       'hormone-refractory prostate cancer', 'prostate cancer recurrent',
       'prostate examination abnormal', 'neoplasm prostate'], dtype=object)

In [19]:
prostate_canc=prostate_canc.drop_duplicates()

In [20]:
prostate_canc.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
11798,prostate cancer metastatic,prednisolon,50.58,1.0,100.0,convulsion,Sodium channel protein type 10 subunit alpha,Bupivacaine Action Pathway
11799,prostate cancer metastatic,prednisolon,50.58,1.0,100.0,convulsion,Prostaglandin E2 receptor EP1 subtype,Bupivacaine Action Pathway
11800,prostate cancer metastatic,prednisolon,50.58,1.0,100.0,hepatic failure,Sodium channel protein type 10 subunit alpha,Bupivacaine Action Pathway
11801,prostate cancer metastatic,prednisolon,50.58,1.0,100.0,hepatic failure,Prostaglandin E2 receptor EP1 subtype,Bupivacaine Action Pathway
11982,prostate cancer metastatic,prednisolon,60.43,1.0,78.0,no reaction,Sodium channel protein type 10 subunit alpha,Bupivacaine Action Pathway


In [21]:
prostate_canc.to_csv('prostate_canc.csv', index=False)

In [17]:
prostate_canc['pt'].value_counts()

cerebral ischaemia                        694
no reaction                               431
asthenia                                  179
dyspnoea                                  122
anaemia                                   121
oedema peripheral                         107
pain                                      104
general physical health deterioration     101
dehydration                               100
febrile neutropenia                        96
malignant neoplasm progression             95
pyrexia                                    93
dizziness                                  92
osteonecrosis of jaw                       89
osteonecrosis                              87
decreased appetite                         87
fatigue                                    86
confusional state                          79
prostatic specific antigen increased       78
renal failure                              76
arthralgia                                 70
sepsis                            

In [9]:
df_breastcancer=df4[df4['indi_pt'].str.contains('breast cancer')].drop(['indi_pt'], axis=1).reset_index(drop=True)
df_breastcancer.to_csv('df_breastcancer_WID.csv', index=False)

In [5]:
len(df4)

10755993

In [6]:
dr4_copy=df4.drop_duplicates()
len(dr4_copy)

10106694

In [7]:
dr4_copy['sepsis']=0.0
dr4_copy.loc[dr4_copy['pt']=='sepsis', 'sepsis']=1
dr4_copy=dr4_copy.drop(['pt','drugname'], axis=1)
dr4_copy.head()

,indi_pt,age,gndr_cod,wt,target,pathway,sepsis
0,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
1,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
2,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
3,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
4,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0


In [10]:
dr4_copy['sepsis'].value_counts()

0.0    10062439
1.0       44255
Name: sepsis, dtype: int64

In [13]:
dr5_c=dr4_copy.copy()
dummy_col = pd.get_dummies(dr5_c[['target','pathway','indi_pt']])
dr5_c=dr5_c.drop(['indi_pt','target','pathway'] , axis=1)
dr5_c = pd.concat([dr5_c, dummy_col], axis=1)
X=dr5_c.drop(['sepsis'], axis=1)
y=dr5_c['sepsis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

MemoryError: 

In [12]:
fscore=0
accuracy=0
recall=0
for i in range(1,10):
    df1=dr4_copy[dr4_copy['sepsis']==1]
    df0=dr4_copy[dr4_copy['sepsis']==0]
    df_0_s=df0.sample(n=50000)
    df_t=pd.concat([df1, df_0_s]).sample(frac=1).reset_index(drop=True)
    X=df_t.drop(['sepsis'], axis=1)
    y=df_t['sepsis']
    dummy_col = pd.get_dummies(X[['target','pathway','indi_pt']])
    X=X.drop(['indi_pt','target','pathway'] , axis=1)
    X = pd.concat([X, dummy_col], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
    clf_loop = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=False)    #building 50 decision trees
    clf_loop.fit(X_train, y_train)
    #clf_loop= KNeighborsClassifier(n_neighbors=1)
    #clf_loop.fit(X_train_res, y_train_res)
    fscore=fscore+ metrics.f1_score(y_test, clf_loop.predict(X_test))
    accuracy=accuracy+metrics.accuracy_score(y_test, clf_loop.predict(X_test))
    recall=recall+metrics.recall_score(y_test, clf_loop.predict(X_test))
print "f1 score:", fscore/10
print "accuracy:", accuracy/10
print "recall :", recall/10

f1 score: 0.8475536614717172
accuracy: 0.8506895820072142
recall : 0.8485765928603705


In [3]:
df4['indi_pt'].value_counts()

product used for unknown indication                 2603195
pulmonary arterial hypertension                      592403
osteoporosis                                         548674
drug use for unknown indication                      448412
osteopenia                                           264544
prophylaxis                                          249633
anaemia                                              235857
supplementation therapy                              198599
mineral supplementation                              196717
pain                                                 185194
pulmonary hypertension                               183808
insomnia                                             181741
vitamin supplementation                              176559
cor pulmonale chronic                                145081
depression                                           120534
hypertension                                         109097
sleep disorder                          

In [90]:
df_hyper=df4[df4['indi_pt']=='osteoporosis']
df_hyper=df_hyper.drop(['indi_pt'],axis=1)
df_hyper['pt'].value_counts()

fall                                          7567
femur fracture                                6500
depression                                    5659
osteoarthritis                                5148
anxiety                                       4605
arthralgia                                    4546
no reaction                                   4514
hypertension                                  4342
pain in extremity                             4216
osteonecrosis                                 3976
back pain                                     3962
gastrooesophageal reflux disease              3893
impaired healing                              3728
tooth disorder                                3691
osteonecrosis of jaw                          3534
osteomyelitis                                 3434
fatigue                                       3369
hyperlipidaemia                               3277
nausea                                        3193
arthropathy                    

In [91]:
df_hyper['femur_fracture']=0.0
df_hyper.loc[df_hyper['pt']=='femur fracture', 'femur_fracture']=1
df_hyper=df_hyper.drop(['pt'], axis=1)
df_hyper.head()

,drugname,age,gndr_cod,wt,target,pathway,femur_fracture
0,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
1,boniva,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
2,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
3,boniva,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
4,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0


In [92]:
dummy_col = pd.get_dummies(df_hyper[['target','pathway']])
df_hyper=df_hyper.drop(['drugname','target','pathway'] , axis=1)
df_hyper = pd.concat([df_hyper, dummy_col], axis=1)
df_hyper.head()

,age,gndr_cod,wt,femur_fracture,target_11-cis retinol dehydrogenase,target_2-oxoglutarate and iron-dependent oxygenase domain-containing protein 1,target_2-oxoglutarate and iron-dependent oxygenase domain-containing protein 2,target_3 beta-hydroxysteroid dehydrogenase/Delta 5-->4-isomerase type 1,target_3-phosphoinositide-dependent protein kinase 1,target_5'-methylthioadenosine/S-adenosylhomocysteine nucleosidase,...,"pathway_Valine, Leucine, and Isoleucine Degradation",pathway_Vitamin A Deficiency,pathway_Warburg Effect,pathway_Wolman Disease,pathway_Xanthine Dehydrogenase Deficiency (Xanthinuria),pathway_Xanthinuria Type I,pathway_Xanthinuria Type II,pathway_Xanthophyll Cycle,pathway_Zoledronate Action Pathway,pathway_beta-Ureidopropionase Deficiency
0,72.2,0.0,39.68,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,72.2,0.0,39.68,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,72.2,0.0,39.68,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72.2,0.0,39.68,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72.2,0.0,39.68,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
y=df_hyper['femur_fracture']
X=df_hyper.drop(['femur_fracture'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print("femur_fracture(pt) model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")

femur_fracture(pt) model
oob score: 0.8341656439978453
0.8352409418969162
[[45700  8518]
 [  522   128]]


In [84]:
lr= LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
print (metrics.accuracy_score(y_test, lr.predict(X_test)))
print (metrics.confusion_matrix(y_test, lr.predict(X_test)))
print (metrics.classification_report(y_test, lr.predict(X_test)))
print (metrics.roc_auc_score(y_test, lr.predict(X_test)))

0.5015856236786469
[[27088 27130]
 [  217   433]]
             precision    recall  f1-score   support

        0.0       0.99      0.50      0.66     54218
        1.0       0.02      0.67      0.03       650

avg / total       0.98      0.50      0.66     54868

0.5828832604556534


In [87]:
from imblearn.under_sampling import (ClusterCentroids, RandomUnderSampler,
                                     NearMiss,
                                     InstanceHardnessThreshold,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     RepeatedEditedNearestNeighbours,
                                     AllKNN,
                                     NeighbourhoodCleaningRule,
                                     OneSidedSelection)

In [88]:
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=0)
ru=RandomUnderSampler(random_state=0)
X_resampled, y_resampled = ru.fit_sample(X_train, y_train)
#X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_resampled, y_resampled, test_size=0.1, random_state=0, stratify=y)
clf2 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf2.fit(X_resampled, y_resampled)
print "femur_fracture(pt) model (UNDERSAMPLING)"
print "oob score:", clf2.oob_score_
print metrics.accuracy_score(y_test, clf2.predict(X_test))

print metrics.confusion_matrix(y_test, clf2.predict(X_test))

femur_fracture(pt) model (UNDERSAMPLING)
oob score: 0.5629059829059829
0.5675803747175038
[[30807 23411]
 [  313   337]]


In [89]:
from imblearn.over_sampling import SMOTE  
#imblearn.over_sampling.SMOTE
sme = SMOTE(random_state=42)
X_res, y_res = sme.fit_sample(X_train, y_train)
clf3 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf3.fit(X_res, y_res)
print "femur_fracture(pt) model (OVERSAMPLING)"
print "oob score:", clf3.oob_score_
print metrics.accuracy_score(y_test, clf3.predict(X_test))

print metrics.confusion_matrix(y_test, clf3.predict(X_test))

femur_fracture(pt) model (OVERSAMPLING)
oob score: 0.969132462763036
0.964861121236422
[[52915  1303]
 [  625    25]]


In [94]:
df4['pt'].value_counts()

no reaction                            364703
nausea                                 139156
dyspnoea                               125077
diarrhoea                              122694
pain                                   108231
fatigue                                 97553
vomiting                                96094
headache                                92933
drug ineffective                        87156
pneumonia                               87004
dizziness                               86980
pyrexia                                 73077
fall                                    70243
anaemia                                 69388
weight decreased                        65472
anxiety                                 64827
hypotension                             63520
malaise                                 59556
asthenia                                57289
arthralgia                              57253
pain in extremity                       57203
back pain                         

In [109]:
df5=df4.copy()

In [110]:
df5['sepsis']=0.0
df5.loc[df5['pt']=='sepsis', 'sepsis']=1
df5=df5.drop(['pt','drugname'], axis=1)
df5.head()

,indi_pt,age,gndr_cod,wt,target,pathway,sepsis
0,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
1,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
2,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
3,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
4,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0


In [111]:
df5['indi_pt'].value_counts()

product used for unknown indication                 2603195
pulmonary arterial hypertension                      592403
osteoporosis                                         548674
drug use for unknown indication                      448412
osteopenia                                           264544
prophylaxis                                          249633
anaemia                                              235857
supplementation therapy                              198599
mineral supplementation                              196717
pain                                                 185194
pulmonary hypertension                               183808
insomnia                                             181741
vitamin supplementation                              176559
cor pulmonale chronic                                145081
depression                                           120534
hypertension                                         109097
sleep disorder                          

In [112]:
df5.loc[df5['indi_pt']=='product used for unknown indication', 'indi_pt']=None
df5.loc[df5['indi_pt']=='drug use for unknown indication', 'indi_pt']=None
len(df5)

10755993

In [113]:
df6=df5.dropna().reset_index(drop=True)
len(df6)

7704386

In [147]:
import gc
gc.collect()

351

In [115]:
df6=df6.drop_duplicates()
len(df6)

2114422

In [116]:
df6['sepsis'].value_counts()

0.0    2087693
1.0      26729
Name: sepsis, dtype: int64

In [120]:
df6_1=df6[df6['sepsis']==1]
df6_0=df6[df6['sepsis']==0]
df6_0_s=df6_0.sample(n=27000)
df7=pd.concat([df6_1, df6_0_s]).sample(frac=1).reset_index(drop=True)
df7.head()

,indi_pt,age,gndr_cod,wt,target,pathway,sepsis
0,colon cancer,73.00,1.0,83.0,DNA,Capecitabine Action Pathway,1.0
1,intraocular lens implant,82.00,0.0,70.0,Cytochrome P450 3A4,Fenoprofen Action Pathway,0.0
2,pulmonary arterial hypertension,61.00,1.0,95.0,Sodium- and chloride-dependent glycine transpo...,Arginine: Glycine Amidinotransferase Deficienc...,0.0
3,pulmonary hypertension,12.17,0.0,37.0,Glycine N-acyltransferase,"Creatine Deficiency, Guanidinoacetate Methyltr...",1.0
4,fluid replacement,85.00,1.0,71.0,Gamma-aminobutyric acid receptor subunit gamma-3,Pyruvate Metabolism,1.0


In [121]:
dummy_col = pd.get_dummies(df7[['indi_pt','target','pathway']])
df7=df7.drop(['indi_pt','target','pathway'] , axis=1)
df7 = pd.concat([df7, dummy_col], axis=1)
df7.head()

,age,gndr_cod,wt,sepsis,indi_pt_abdominal discomfort,indi_pt_abdominal distension,indi_pt_abdominal pain,indi_pt_abdominal pain lower,indi_pt_abdominal pain upper,indi_pt_abdominal rigidity,...,pathway_Ximelagatran Action Pathway,pathway_Zidovudine Action Pathway,pathway_Zoledronate Action Pathway,pathway_beta-Alanine Metabolism,pathway_beta-Mercaptolactate-Cysteine Disulfiduria,pathway_beta-Ureidopropionase Deficiency,pathway_gamma-Glutamyltransferase Deficiency,pathway_gamma-Glutamyltranspeptidase Deficiency,pathway_tRNA Charging,pathway_tRNA Charging 2
0,73.00,1.0,83.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,82.00,0.0,70.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,61.00,1.0,95.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,12.17,0.0,37.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,85.00,1.0,71.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
y=df7['sepsis']
X=df7.drop(['sepsis'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print("sepsis(pt) model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")

sepsis(pt) model
oob score: 0.9588882455124493
0.9597989949748744
[[2579  121]
 [  95 2578]]


In [150]:
df8=df4.copy()
df8.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
0,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
1,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Hydroxylapatite,Ibandronate Action Pathway
2,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
3,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Hydroxylapatite,Ibandronate Action Pathway
4,osteoporosis,boniva,72.2,0.0,39.68,vomiting,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [137]:
df8['pt'].value_counts()

no reaction                            364703
nausea                                 139156
dyspnoea                               125077
diarrhoea                              122694
pain                                   108231
fatigue                                 97553
vomiting                                96094
headache                                92933
drug ineffective                        87156
pneumonia                               87004
dizziness                               86980
pyrexia                                 73077
fall                                    70243
anaemia                                 69388
weight decreased                        65472
anxiety                                 64827
hypotension                             63520
malaise                                 59556
asthenia                                57289
arthralgia                              57253
pain in extremity                       57203
back pain                         

In [151]:
df8['pneumonia']=0.0
df8.loc[df8['pt']=='pneumonia', 'pneumonia']=1
df8=df8.drop(['pt','drugname'], axis=1)
df8.head()

,indi_pt,age,gndr_cod,wt,target,pathway,pneumonia
0,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
1,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
2,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
3,osteoporosis,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
4,osteoporosis,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0


In [152]:
df8.loc[df8['indi_pt']=='product used for unknown indication', 'indi_pt']=None
df8.loc[df8['indi_pt']=='drug use for unknown indication', 'indi_pt']=None
len(df8)

10755993

In [153]:
df8=df8.dropna().reset_index(drop=True)
len(df8)

7704386

In [154]:
df8=df8.drop_duplicates()
len(df8)

2133957

In [142]:
df8['pneumonia'].value_counts()

0.0    2083153
1.0      50804
Name: pneumonia, dtype: int64

In [155]:
df8_1=df8[df8['pneumonia']==1]
df8_0=df8[df8['pneumonia']==0]
df8_0_s=df8_0.sample(n=51000)
df8=pd.concat([df8_1, df8_0_s]).sample(frac=1).reset_index(drop=True)
df8.head()

,indi_pt,age,gndr_cod,wt,target,pathway,pneumonia
0,pulmonary arterial hypertension,41.0,0.0,91.0,Glycine N-acyltransferase-like protein 1,Hyperornithinemia with Gyrate Atrophy (HOGA),1.0
1,breakthrough pain,52.0,1.0,59.0,Lymphocyte antigen 96,Methadone Action Pathway,1.0
2,blood potassium,75.0,0.0,90.0,Cationic amino acid transporter 3,Inner Membrane Transport,1.0
3,pulmonary arterial hypertension,41.0,0.0,91.0,"Glycine cleavage system H protein, mitochondrial",Glutathione Metabolism II,1.0
4,deep vein thrombosis,84.0,0.0,44.0,Nuclear receptor subfamily 1 group I member 2,Warfarin Action Pathway,0.0


In [156]:
dummy_col = pd.get_dummies(df8[['indi_pt','target','pathway']])
df8=df8.drop(['indi_pt','target','pathway'] , axis=1)
df8 = pd.concat([df8, dummy_col], axis=1)
df8.head()

,age,gndr_cod,wt,pneumonia,indi_pt_abdominal discomfort,indi_pt_abdominal distension,indi_pt_abdominal hernia repair,indi_pt_abdominal operation,indi_pt_abdominal pain,indi_pt_abdominal pain lower,...,pathway_Ximelagatran Action Pathway,pathway_Zidovudine Action Pathway,pathway_Zoledronate Action Pathway,pathway_beta-Alanine Metabolism,pathway_beta-Mercaptolactate-Cysteine Disulfiduria,pathway_beta-Ureidopropionase Deficiency,pathway_gamma-Glutamyltransferase Deficiency,pathway_gamma-Glutamyltranspeptidase Deficiency,pathway_tRNA Charging,pathway_tRNA Charging 2
0,41.0,0.0,91.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,52.0,1.0,59.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,75.0,0.0,90.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,41.0,0.0,91.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,84.0,0.0,44.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [157]:
y=df8['pneumonia']
X=df8.drop(['pneumonia'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print("pneumonia(pt) model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")


pneumonia(pt) model
oob score: 0.9201401394846273
0.9269226991454671
[[4593  507]
 [ 237 4844]]


In [158]:
lr= LogisticRegression(class_weight='balanced')
lr.fit(X_train, y_train)
print (metrics.accuracy_score(y_test, lr.predict(X_test)))
print (metrics.confusion_matrix(y_test, lr.predict(X_test)))
print (metrics.classification_report(y_test, lr.predict(X_test)))
print (metrics.roc_auc_score(y_test, lr.predict(X_test)))

0.7866614281504763
[[3830 1270]
 [ 902 4179]]
             precision    recall  f1-score   support

        0.0       0.81      0.75      0.78      5100
        1.0       0.77      0.82      0.79      5081

avg / total       0.79      0.79      0.79     10181

0.7867281413647923


In [159]:
df9=df4.copy()
df9.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
0,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
1,osteoporosis,boniva,72.2,0.0,39.68,incorrect drug administration duration,Hydroxylapatite,Ibandronate Action Pathway
2,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
3,osteoporosis,boniva,72.2,0.0,39.68,product odour abnormal,Hydroxylapatite,Ibandronate Action Pathway
4,osteoporosis,boniva,72.2,0.0,39.68,vomiting,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [160]:
df9.loc[df9['indi_pt']=='product used for unknown indication', 'indi_pt']=None
df9.loc[df9['indi_pt']=='drug use for unknown indication', 'indi_pt']=None
df9=df9.dropna().reset_index(drop=True)
df9=df9.drop_duplicates()

In [161]:
df9['pneumonia']=0.0
df9.loc[df9['pt']=='pneumonia', 'pneumonia']=1
df9=df9.drop(['pt'], axis=1)
df9.head()

,indi_pt,drugname,age,gndr_cod,wt,target,pathway,pneumonia
0,osteoporosis,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
1,osteoporosis,boniva,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
2,osteoporosis,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0
3,osteoporosis,boniva,72.2,0.0,39.68,Hydroxylapatite,Ibandronate Action Pathway,0.0
4,osteoporosis,boniva,72.2,0.0,39.68,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway,0.0


In [165]:
df9['pneumonia'].value_counts()

0.0    52000
1.0    51071
Name: pneumonia, dtype: int64

In [163]:
df9_1=df9[df9['pneumonia']==1]
df9_0=df9[df9['pneumonia']==0]
df9_0_s=df9_0.sample(n=52000)
df9=pd.concat([df9_1, df9_0_s]).sample(frac=1).reset_index(drop=True)
df9.head()

,indi_pt,drugname,age,gndr_cod,wt,target,pathway,pneumonia
0,prophylaxis,ascorbic acid,65.00,0.0,74.00,Prolyl 3-hydroxylase 3,Gibberellin Biosynthesis II (Early C-13 Hydrox...,0.0
1,dystonia,magnesium,43.95,0.0,77.55,Alpha-ketoglutarate-dependent dioxygenase alkB...,Gibberellin Biosynthesis II (Early C-13 Hydrox...,0.0
2,prophylaxis against gastrointestinal ulcer,famotidine,6.00,1.0,18.00,Histamine H2 receptor,Famotidine Action Pathway,0.0
3,nausea,novamin,64.00,1.0,55.00,Histidine ammonia-lyase,Methacycline Action Pathway,0.0
4,glossitis,zinc,59.00,0.0,86.20,"Methylcrotonoyl-CoA carboxylase subunit alpha,...",Citric Acid Cycle,0.0


In [164]:
dummy_col = pd.get_dummies(df9[['drugname','indi_pt','target','pathway']])
df9=df9.drop(['drugname','indi_pt','target','pathway'] , axis=1)
df9 = pd.concat([df9, dummy_col], axis=1)
df9.head()

,age,gndr_cod,wt,pneumonia,drugname_3tc,drugname_50% dextrose,drugname_a-hydrocort,drugname_abacavir,drugname_abraxane,drugname_acc,...,pathway_Ximelagatran Action Pathway,pathway_Zidovudine Action Pathway,pathway_Zoledronate Action Pathway,pathway_beta-Alanine Metabolism,pathway_beta-Mercaptolactate-Cysteine Disulfiduria,pathway_beta-Ureidopropionase Deficiency,pathway_gamma-Glutamyltransferase Deficiency,pathway_gamma-Glutamyltranspeptidase Deficiency,pathway_tRNA Charging,pathway_tRNA Charging 2
0,65.00,0.0,74.00,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,43.95,0.0,77.55,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6.00,1.0,18.00,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,64.00,1.0,55.00,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,59.00,0.0,86.20,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
y=df9['pneumonia']
X=df9.drop(['pneumonia'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print("pneumonia(pt) model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")

pneumonia(pt) model
oob score: 0.8910664812479113
0.8925106713232441
[[4577  623]
 [ 485 4623]]


In [179]:
df10=df4.copy()
df10.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
0,osteopenia,boniva,59.0,0.0,48.534344,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
1,osteopenia,boniva,59.0,0.0,48.534344,0,Hydroxylapatite,Ibandronate Action Pathway
2,prophylaxis,boniva,61.0,0.0,95.240000,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
3,prophylaxis,boniva,61.0,0.0,95.240000,0,Hydroxylapatite,Ibandronate Action Pathway
4,osteopenia,boniva,58.0,0.0,63.500000,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [180]:
df10.loc[df10['indi_pt']=='product used for unknown indication', 'indi_pt']=None
df10.loc[df10['indi_pt']=='drug use for unknown indication', 'indi_pt']=None
df10=df10.dropna()
df10=df10.drop_duplicates().reset_index(drop=True)

In [181]:
df10_0=df10[df10['pt']=='dyspnoea']
df10_1=df10[df10['pt']=='diarrhoea']
df10_2=df10[df10['pt']=='arthralgia']
df10_3=df10[df10['pt']=='dizziness']
df10_4=df10[df10['pt']=='pneumonia']
df10_5=df10[df10['pt']=='pyrexia']
df10_6=df10[df10['pt']=='weight decreased']
df10_7=df10[df10['pt']=='anaemia']
df10_8=df10[df10['pt']=='anxiety']
df10_0['pt']=0
df10_1['pt']=1
df10_2['pt']=2
df10_3['pt']=3
df10_4['pt']=4
df10_5['pt']=5
df10_6['pt']=6
df10_7['pt']=7
df10_8['pt']=8
df3=pd.concat([df10_0, df10_1, df10_2, df10_3, df10_4, df10_5, df10_6, df10_7, df10_8])
df3.head()

TypeError: invalid type comparison

In [172]:
dummy_col = pd.get_dummies(df3[['drugname','indi_pt','target','pathway']])
df4=df3.drop(['drugname','indi_pt','target','pathway'] , axis=1)
df4 = pd.concat([df4, dummy_col], axis=1)
df4.head()

,age,gndr_cod,wt,pt,drugname_3tc,drugname_a + d ointment,drugname_abacavir,drugname_abacavir/lamivudine,drugname_abraxane,drugname_abstral,...,pathway_Ximelagatran Action Pathway,pathway_Zidovudine Action Pathway,pathway_Zoledronate Action Pathway,pathway_beta-Alanine Metabolism,pathway_beta-Mercaptolactate-Cysteine Disulfiduria,pathway_beta-Ureidopropionase Deficiency,pathway_gamma-Glutamyltransferase Deficiency,pathway_gamma-Glutamyltranspeptidase Deficiency,pathway_tRNA Charging,pathway_tRNA Charging 2
60,59.0,0.0,48.534344,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
61,59.0,0.0,48.534344,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
262,61.0,0.0,95.240000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
263,61.0,0.0,95.240000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
306,58.0,0.0,63.500000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [173]:
y=df4['pt']
X=df4.drop(['pt'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)
print("pt model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")

KeyboardInterrupt: 

In [3]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

In [18]:
df44=pd.read_csv('df_dr_reac.csv')

In [33]:
df2=df44.copy()

In [34]:
df2.loc[df2['indi_pt']=='product used for unknown indication', 'indi_pt']=None
df2.loc[df2['indi_pt']=='drug use for unknown indication', 'indi_pt']=None
df2=df2.dropna()
df2=df2.drop_duplicates().reset_index(drop=True)
df2=df2[df2['pt']!='no reaction'].reset_index(drop=True)

In [35]:
df2_0=df2[df2['pt']=='hypertension']
df2_1=df2[df2['pt']=='hypotension']
df2_2=df2[df2['pt']=='dehydration']
df2_3=df2[df2['pt']=='dizziness']
df2_4=df2[df2['pt']=='pneumonia']
df2_5=df2[df2['pt']=='pyrexia']
df2_6=df2[df2['pt']=='weight decreased']
df2_7=df2[df2['pt']=='anaemia']
df2_0['pt']=0
df2_1['pt']=1
df2_2['pt']=2
df2_3['pt']=3
df2_4['pt']=4
df2_5['pt']=5
df2_6['pt']=6
df2_7['pt']=7
df3=pd.concat([df2_0, df2_1, df2_2, df2_3, df2_4, df2_5, df2_6, df2_7])
#df3=pd.concat([df2_0, df2_1,df2_2, df2_3, df2_4]).reset_index(drop=True)
df3.head()

,indi_pt,drugname,age,gndr_cod,wt,pt,target,pathway
8,osteoporosis,bonviva,57.0,0.0,65.0,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
9,osteoporosis,bonviva,57.0,0.0,65.0,0,Hydroxylapatite,Ibandronate Action Pathway
298,osteoporosis,boniva,49.0,0.0,84.8,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway
299,osteoporosis,boniva,49.0,0.0,84.8,0,Hydroxylapatite,Ibandronate Action Pathway
404,osteitis deformans,boniva,79.0,0.0,62.6,0,Farnesyl pyrophosphate synthase,Ibandronate Action Pathway


In [36]:
dummy_col = pd.get_dummies(df3[['indi_pt','target','pathway']])
df4=df3.drop(['drugname','indi_pt','target','pathway'] , axis=1)
df4 = pd.concat([df4, dummy_col], axis=1)
df4.head()

,age,gndr_cod,wt,pt,indi_pt_abdominal discomfort,indi_pt_abdominal distension,indi_pt_abdominal infection,indi_pt_abdominal pain,indi_pt_abdominal pain upper,indi_pt_abnormal behaviour,...,pathway_Xanthinuria Type II,pathway_Xanthophyll Cycle,pathway_Ximelagatran Action Pathway,pathway_Zidovudine Action Pathway,pathway_Zoledronate Action Pathway,pathway_beta-Alanine Metabolism,pathway_beta-Mercaptolactate-Cysteine Disulfiduria,pathway_beta-Ureidopropionase Deficiency,pathway_tRNA Charging,pathway_tRNA Charging 2
8,57.0,0.0,65.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,57.0,0.0,65.0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
298,49.0,0.0,84.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
299,49.0,0.0,84.8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,79.0,0.0,62.6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
df4['pt'].value_counts()

3    62788
4    51071
5    48219
7    40994
6    40352
0    33410
1    32120
2    29421
Name: pt, dtype: int64

In [38]:
y=df4['pt']
X=df4.drop(['pt'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0, stratify=y)
clf1 = RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=-1, random_state=0, oob_score=True)    #building 50 decision trees
clf1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=-1, oob_score=True, random_state=0,
            verbose=0, warm_start=False)

In [39]:
print("pt model")
print("oob score:"), clf1.oob_score_
print (metrics.accuracy_score(y_test, clf1.predict(X_test)))

print (metrics.confusion_matrix(y_test, clf1.predict(X_test)))
print ("=======================================================")

pt model
oob score: 0.6436492117542367
0.6504226018086176
[[1901   86   83  327  306  107  316  215]
 [  86 2115  259  145   93  103  225  186]
 [ 104  279 1789   79  201  151  140  199]
 [ 381  197  141 4578  407  248  175  152]
 [ 361  139  278  355 3003  485  286  200]
 [ 106  118  174  259  489 3341  134  201]
 [ 400  233  185  178  236  142 2484  177]
 [ 237  193  235  121  158  183  175 2798]]
